In [40]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [3]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2021-01.parquet -O  fhv_tripdata_2021-01.parquet
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2021-02.parquet -O  fhv_tripdata_2021-02.parquet

--2022-05-24 03:40:50--  https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.226.67
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.226.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-01.parquet’

fhv_tripdata_2021-0 100%[===================>]  11.33M  5.67MB/s    in 2.0s    

2022-05-24 03:40:53 (5.67 MB/s) - ‘fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]

--2022-05-24 03:40:53--  https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2021-02.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.226.67
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.226.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10645466 (10M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-02.parquet’

fhv_tripdata_2021-0 100%[===================>]  10.15M  6.18MB/s    in 

### Read data and calculate duration 

In [4]:
df = pd.read_parquet('fhv_tripdata_2021-01.parquet')

df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

### Get total number of records 

In [5]:
df.shape

(1154112, 8)

### Calcuate duration average/ mean

In [6]:
df.duration.mean()

19.1672240937939

In [8]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667


In [17]:
clean_df = df[(df.duration >= 1) & (df.duration <= 60)]

### Number of dropped record 

In [9]:
(len(df)-len(clean_df))

44286

### Number of misssing values

In [18]:
percent_missing = clean_df.isnull().sum() * 100 / len(clean_df)
missing_value_df = pd.DataFrame({'column_name': clean_df.columns,
                                 'percent_missing': percent_missing})

In [19]:
missing_value_df

,column_name,percent_missing
dispatching_base_num,dispatching_base_num,0.000000
pickup_datetime,pickup_datetime,0.000000
dropOff_datetime,dropOff_datetime,0.000000
PUlocationID,PUlocationID,83.527328
DOlocationID,DOlocationID,13.327044
SR_Flag,SR_Flag,100.000000
Affiliated_base_number,Affiliated_base_number,0.069651
duration,duration,0.000000


In [20]:
clean_df['PUlocationID']= clean_df.PUlocationID.fillna(-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
percent_missing = clean_df.isnull().sum() * 100 / len(clean_df)
missing_value_df = pd.DataFrame({'column_name': clean_df.columns,
                                 'percent_missing': percent_missing})

In [22]:
missing_value_df

,column_name,percent_missing
dispatching_base_num,dispatching_base_num,0.000000
pickup_datetime,pickup_datetime,0.000000
dropOff_datetime,dropOff_datetime,0.000000
PUlocationID,PUlocationID,0.000000
DOlocationID,DOlocationID,13.327044
SR_Flag,SR_Flag,100.000000
Affiliated_base_number,Affiliated_base_number,0.069651
duration,duration,0.000000


In [24]:
categorical = ['PUlocationID', 'DOlocationID']
numerical = ['trip_distance']

clean_df[categorical] = clean_df[categorical].astype(str)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [29]:
train_dicts = clean_df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)



In [35]:
X_train.shape

(1109826, 525)

In [36]:
from sklearn.metrics import mean_squared_error

In [37]:
target = 'duration'
y_train = clean_df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)



10.528519107211812

In [39]:
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
rmse

10.528519107211812